<a href="https://colab.research.google.com/github/AgustinCocciardi/IA-Aplicada/blob/main/05_Representaci%C3%B3n_de_texto_Enfoques_b%C3%A1sicos_de_representacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Representación de texto: Enfoques básicos de representacion


## One Hot encoding

### Implementación Manual One Hot Encoding

Tomemos los documentos que vimos en la ppt sin stopwords y hagamos un preprocesamiento básico. </br>
1 - Pasemos todo a lowercase. </br>
2 - Quitemos los puntos. </br>

In [ ]:
documents = ["estudiante cursa NLP.", "estudiante rinde examen."]
processed_docs = [doc.lower().replace(".","") for doc in documents]
processed_docs

['estudiante cursa nlp', 'estudiante rinde examen']

Asignemosle un índice a cada una de las palabras de nuestro vocabulario

In [ ]:
vocab = {}
count = 0
for doc in processed_docs:
    for word in doc.split():
        if word not in vocab:
            count = count +1
            vocab[word] = count
print(vocab)

{'estudiante': 1, 'cursa': 2, 'nlp': 3, 'rinde': 4, 'examen': 5}


In [ ]:
#Representamos cualquier string como one hot encoding Basado en las palabras de nuestro vocabulario.
#Si la palabra existe en nuestro vocabulario, se devuelve su representación.
#Si no esta en la lista se devuelve un vector con ceros.
def get_onehot_vector(somestring):
    onehot_encoded = []
    for word in somestring.split():
        temp = [0]*len(vocab)
        if word in vocab:
            temp[vocab[word]-1] = 1 # -1 es porque en python los arrays comienzan en 0
        onehot_encoded.append(temp)
    return onehot_encoded

In [ ]:
print(processed_docs[1])
get_onehot_vector(processed_docs[1]) #one hot representation para un texto de nuestro corpus.

estudiante rinde examen


[[1, 0, 0, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]]

### One-hot encoding utilizando scikit-learn

In [ ]:
S1 = 'estudiante cursa nlp'
S2 = 'estudiante rinde examen'

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

data = [S1.split(), S2.split()]
values = data[0] + data[1]
print("The data: ",values)

#Label Encoding
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print("Label Encoded:",integer_encoded)

#One-Hot Encoding
onehot_encoder = OneHotEncoder()
onehot_encoded = onehot_encoder.fit_transform(data).toarray()
print("Onehot Encoded Matrix:\n",onehot_encoded)

The data:  ['estudiante', 'cursa', 'nlp', 'estudiante', 'rinde', 'examen']
Label Encoded: [1 0 3 1 4 2]
Onehot Encoded Matrix:
 [[1. 1. 0. 0. 1.]
 [1. 0. 1. 1. 0.]]


## Bag of Words

In [ ]:
documents = ["estudiante cursa NLP.", "estudiante rinde examen."]
processed_docs = [doc.lower().replace(".","") for doc in documents]
processed_docs

['estudiante cursa nlp', 'estudiante rinde examen']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#Revisemos la lista de los documentos
print("Nuestro corpus: ", processed_docs)

count_vect = CountVectorizer()
#Creamos una representacion BOW para el corpus
bow_rep = count_vect.fit_transform(processed_docs)

#Miramos el vocabulary mapping del vectorizer
print("Nuestro vocabulario: ", count_vect.vocabulary_)

#Miremos la representación BOW para los primeros 2 documentos
print("Representacion BoW para 'estudiante cursa nlp': ", bow_rep[0].toarray())
print("Representacion BoW para 'estudiante rinde examen': ",bow_rep[1].toarray())

#Obtengamos una nueva representación usando este vocabulario, para un nuevo texto
temp = count_vect.transform(["estudiante rinde nlp"])
print("Representacion BoW para 'estudiante rinde nlp':", temp.toarray())

#Veamos si hay palabras duplicadas:
temp = count_vect.transform(["estudiante rinde nlp nlp"])
print("Representacion BoW para 'estudiante rinde nlp nlp':", temp.toarray())

Nuestro corpus:  ['estudiante cursa nlp', 'estudiante rinde examen']
Nuestro vocabulario:  {'estudiante': 1, 'cursa': 0, 'nlp': 3, 'rinde': 4, 'examen': 2}
Representacion BoW para 'estudiante cursa nlp':  [[1 1 0 1 0]]
Representacion BoW para 'estudiante rinde examen':  [[0 1 1 0 1]]
Representacion BoW para 'estudiante rinde nlp': [[0 1 0 1 1]]
Representacion BoW para 'estudiante rinde nlp nlp': [[0 1 0 2 1]]


In [ ]:
#BoW con vectores binarios. Estos se usaban en tareas de analisis de sentimientos que no necesita saber la cantidad de veces que se repite una palabra sino su mera presencia.
count_vect_bin = CountVectorizer(binary=True)
count_vect_bin.fit(processed_docs)
temp = count_vect_bin.transform(["estudiante rinde nlp"])
print("Representacion BoW binario para 'estudiante rinde nlp':", temp.toarray())

Representacion BoW binario para 'estudiante rinde nlp': [[0 1 0 1 1]]


In [ ]:
# Comparamos resultados entre representaciones binarias y normales(Frecuencias)
temp = count_vect.transform(["estudiante rinde nlp nlp"])
print("Representacion BoW para 'estudiante rinde nlp nlp':", temp.toarray())
temp = count_vect_bin.transform(["estudiante rinde nlp nlp"])
print("Representacion BoW binario para 'estudiante rinde nlp':", temp.toarray())

Representacion BoW para 'estudiante rinde nlp nlp': [[0 1 0 2 1]]
Representacion BoW binario para 'estudiante rinde nlp': [[0 1 0 1 1]]


## Bag of N-grams

Vamos a capturar un poco de contexto usando N-grams. Esto nos permite capturar algunas relaciones entre palabras.

In [ ]:
documents = ["estudiante cursa NLP.", "estudiante rinde examen."]
processed_docs = [doc.lower().replace(".","") for doc in documents]
processed_docs

['estudiante cursa nlp', 'estudiante rinde examen']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#Vectorización Ngram con count vectorizer desde uni, bi hasta trigrams
count_vect = CountVectorizer(ngram_range=(1,3))

#Contruimos una representación BOW para el corpus
bow_rep = count_vect.fit_transform(processed_docs)

#Miremos como queda el vocabulary mapping
print("Nuestro vocabulario: ", count_vect.vocabulary_)

#Miremos la representación BOW de los primeros 2 documents
print("Representacion BoW  para 'estudiante cursa nlp': ", bow_rep[0].toarray())
print("Representacion BoW  para 'estudiante rinde examen': ",bow_rep[1].toarray())

#Armemos la representación de un nuevo texto usando este vocabulario
temp = count_vect.transform(["estudiante rinde nlp"])
print("Representación BoN  para  'estudiante rinde nlp':", temp.toarray())

Nuestro vocabulario:  {'estudiante': 2, 'cursa': 0, 'nlp': 8, 'estudiante cursa': 3, 'cursa nlp': 1, 'estudiante cursa nlp': 4, 'rinde': 9, 'examen': 7, 'estudiante rinde': 5, 'rinde examen': 10, 'estudiante rinde examen': 6}
Representacion BoW  para 'estudiante cursa nlp':  [[1 1 1 1 1 0 0 0 1 0 0]]
Representacion BoW  para 'estudiante rinde examen':  [[0 0 1 0 0 1 1 1 0 1 1]]
Representación BoN  para  'estudiante rinde nlp': [[0 0 1 0 0 1 0 0 1 1 0]]



## TF-IDF

En todos los demás enfoques que hemos visto hasta ahora, todas las palabras del texto se tratan con la misma importancia. No existe la noción de que algunas palabras del documento sean más importantes que otras. TF-IDF se aborda esta problemática. Su objetivo es cuantificar la importancia de una palabra determinada en relación con otras palabras en el documento y en el corpus. Era un esquema de representación comúnmente utilizado para sistemas de recuperación de información, para extraer documentos relevantes de un corpus para una consulta de texto determinada.

In [ ]:
documents = ["estudiante cursa NLP.", "estudiante rinde examen.", "profesor viaja universidad", "estudiante cursa virtual"]
processed_docs = [doc.lower().replace(".","") for doc in documents]
processed_docs

['estudiante cursa nlp',
 'estudiante rinde examen',
 'profesor viaja universidad',
 'estudiante cursa virtual']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
bow_rep_tfidf = tfidf.fit_transform(processed_docs)

print("IDF de todas las palabras en el vocabulario",tfidf.idf_)
print("-"*10)

print("Todas las palabras en el vocabulario",tfidf.get_feature_names_out())
print("-"*10)

print("Representacion TFIDF para todos documentos en nuestro corpus\n",bow_rep_tfidf.toarray())
print("-"*10)

# Representación TFIDF para un texto nuevo que usa nuestro vocabulario.

temp = tfidf.transform(["estudiante rinde nlp"])
print("Representacion TFIDF para 'estudiante rinde nlp':\n", temp.toarray())

IDF de todas las palabras en el vocabulario [1.51082562 1.22314355 1.91629073 1.91629073 1.91629073 1.91629073
 1.91629073 1.91629073 1.91629073]
----------
Todas las palabras en el vocabulario ['cursa' 'estudiante' 'examen' 'nlp' 'profesor' 'rinde' 'universidad'
 'viaja' 'virtual']
----------
Representacion TFIDF para todos documentos en nuestro corpus
 [[0.55349232 0.44809973 0.         0.70203482 0.         0.
  0.         0.         0.        ]
 [0.         0.41137791 0.64450299 0.         0.         0.64450299
  0.         0.         0.        ]
 [0.         0.         0.         0.         0.57735027 0.
  0.57735027 0.57735027 0.        ]
 [0.55349232 0.44809973 0.         0.         0.         0.
  0.         0.         0.70203482]]
----------
Representacion TFIDF para 'estudiante rinde nlp':
 [[0.         0.41137791 0.         0.64450299 0.         0.64450299
  0.         0.         0.        ]]


Aplicar una técnica a un texto pequeño y uno grande.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Textos a representar
small_text = 'ayer fue viernes'
large_text = 'el jueves pasado se ha disputado un partido de futbol entre el club atletico boca juniors y el cruzeiro esporte clube resultando en victoria por un solo gol para el equipo local cuyo tanto fue anotado por el delantero edinson cavani'

# Crear un vectorizador BoW
vectorizer = CountVectorizer()

# Ajustar el vectorizador a los textos
vectorizer.fit([small_text, large_text])

# Transformar los textos en vectores separados
small_text_vector = vectorizer.transform([small_text]).toarray()
large_text_vector = vectorizer.transform([large_text]).toarray()

# Mostrar el vocabulario
print("Vocabulario:", vectorizer.get_feature_names_out())

# Mostrar la representación BoW de small_text
print("Vector BoW para small_text: ", small_text)
print(small_text_vector)

# Mostrar la representación BoW de large_text
print("Vector BoW para large_text:", large_text)
print(large_text_vector)

Vocabulario: ['anotado' 'atletico' 'ayer' 'boca' 'cavani' 'club' 'clube' 'cruzeiro'
 'cuyo' 'de' 'delantero' 'disputado' 'edinson' 'el' 'en' 'entre' 'equipo'
 'esporte' 'fue' 'futbol' 'gol' 'ha' 'jueves' 'juniors' 'local' 'para'
 'partido' 'pasado' 'por' 'resultando' 'se' 'solo' 'tanto' 'un' 'victoria'
 'viernes']
Vector BoW para small_text:  ayer fue viernes
[[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]]
Vector BoW para large_text: el jueves pasado se ha disputado un partido de futbol entre el club atletico boca juniors y el cruzeiro esporte clube resultando en victoria por un solo gol para el equipo local cuyo tanto fue anotado por el delantero edinson cavani
[[1 1 0 1 1 1 1 1 1 1 1 1 1 5 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 2 1 0]]
